# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [104]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as ols

df = pd.read_csv('homepage_actions.csv')

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [106]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [107]:
control = df[df['group'] == 'control']['id']
experiment = df[df['group'] == 'experiment']['id']

len(np.intersect1d(control, experiment))
# no overlap between groups

0

In [108]:
clicked = set(df[df['action'] == 'click']['id'].unique())
viewed = set(df[df['action'] == 'view']['id'].unique())

len(viewed-clicked) == (len(viewed) - len(clicked))
# everyone that clicked also viewed

True

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [109]:
cont_vc = df[df['group'] == 'control']['action'].value_counts()
cont_click_rate = cont_vc[1] / cont_vc[0]

exp_vc = df[df['group'] == 'experiment']['action'].value_counts()
exp_click_rate = exp_vc[1] / exp_vc[0]

print(cont_click_rate, exp_click_rate)

0.2797118847539016 0.3097463284379172


In [120]:
from scipy import stats

df['count'] = 1
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

stats.ttest_ind(control.click, experiment.click, equal_var = True)[1]/2

0.0044125494574791466

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [127]:
cont_rate = control['click'].mean()
expected = cont_rate * len(experiment)
print(expected)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [129]:
actual = experiment['click'].sum()
var = len(experiment) * cont_rate * (1-cont_rate)
z = (actual - expected)/ np.sqrt(var)
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [130]:
stats.norm.sf(z)

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **this does roughly match the previous test since both p values are very low. In both cases we would reject the 
null hypothesis. This means that the experimental homepage is more effective at getting clicks**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.